In [25]:
import os
import json
import requests
# from serpapi import GoogleSearch
import re
from langchain_groq import ChatGroq
import serpapi
from youtube_transcript_api import YouTubeTranscriptApi
# Initialize LLM with Groq
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.4,
    max_tokens=2048,
    api_key="GROQ_API_KEY"
    
)

def extract_video_id(url):
    match = re.search(r"(?:v=|be/)([a-zA-Z0-9_-]{11})", url)
    return match.group(1) if match else None

def get_video_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        transcript_text = " ".join([entry['text'] for entry in transcript])
        return transcript_text
    except Exception as e:
        return f"Error fetching transcript: {str(e)}"
    
# Function to search YouTube using SerpAPI
def search_youtube(query):
    client = serpapi.Client(api_key=os.getenv("SERPAPI_API_KEY"))

    r = client.search({
        "engine": "youtube",
        "search_query": query,
    })
    

    if "video_results" in r and r["video_results"]:
        top_video = r["video_results"][0]  # Pick the topmost result
        return {
            "title": top_video["title"],
            "link": top_video["link"],
            "description": top_video.get("snippet", "No description available."),
            "thumbnail": top_video.get("thumbnail"),
            "views": top_video.get("views"),
            "length": top_video.get("length"),
            "published_date": top_video.get("published_date"),
        }
    else:
        return None  # No results found


# Function to summarize the video using Llama-3 (Groq)
def summarize_video_with_llm(video_title, video_description, video_link, transcript):
    
    messages = [
        {
            "role": "system",
            "content": "You are an AI assistant that summarizes YouTube videos into structured, immersive, and highly detailed narratives."
        },
        {
            "role": "user",
            "content": f"""
            **Summarize this YouTube video as if the user is watching it in real-time. Make the experience immersive.**  
            
            - Provide a **scene-by-scene breakdown** with timestamps (if available), ensuring that each moment is **richly detailed and vividly described**.  
            - Include **all key characters**, their **roles, relationships, and motivations**.  
            - Describe **visual details** (lighting, colors, expressions, body language, atmosphere, etc.).  
            - Capture **emotions, key dialogues, and dramatic moments** to bring the summary to life.  
            - If there are important **actions or turning points**, emphasize their significance.  
            - Expand on **key themes and deeper insights** separately, using concrete examples from the video.  
            
            The summary should be **thorough, engaging, and as detailed as possible**. Assume the reader has no access to the video and needs a full breakdown.  
            
            **Title:** {video_title}  
            **Description:** {video_description}  
            **Link:** {video_link}  
            **Transcript:** {transcript[:4000]}  
    
            **Example JSON Output Format:**  
            ```json
            {{
                "summary": "The video opens with a dramatic confrontation... [Elaborate with full details]",
                "key_themes": [
                    "Theme 1 - Explained in depth",
                    "Theme 2 - With supporting examples",
                    "Theme 3 - With emotional impact"
                ]
            }}
            ```
            """
        }
    ]
    response = llm.invoke(messages)
    return response  # Groq will return a structured response

def process_llm_response(llm_response):
    json_match = re.search(r"```json\n({.*})\n```", llm_response, re.DOTALL)
    if json_match:
        json_data = json.loads(json_match.group(1))  # Parse as JSON``
        return json_data
    else:
        print("JSON not found in response")
    
# Main function to execute the workflow
def process_video_summary(query):
    video_info = search_youtube(query)
    if not video_info:
        return "No matching videos found."
        
    video_id = extract_video_id(video_info["link"])
    
    transcript = get_video_transcript(video_id)
    llm_summary = summarize_video_with_llm(video_info["title"], video_info["description"], video_info["link"],transcript)
    
    json_data = process_llm_response(llm_summary.content)
        # Extract values
    summary = json_data.get("summary", "")
    key_themes = json_data.get("key_themes", [])
    response = {
        "title": video_info["title"],
        "link": video_info["link"],
        "description": video_info["description"],
        "thumbnail": video_info["thumbnail"],
        # "thumbnail_rich": video_info["thumbnail"]["rich"],
        "views": video_info["views"],
        "length": video_info["length"],
        "summary": summary,
        "key_themes": key_themes,
        # "transcript":transcript,

        # "llm_summary": llm_summary,
    }
    return response  # Groq returns structured JSON response

# Example usage
query = input("Enter the YouTube video title or topic: ")
result = process_video_summary(query)

print("\n**Video Summary:**")
print(result)



**Video Summary:**
{'title': 'LOVE IN EVERY WORD - Nigerian Movies 2025 Latest Full Movies', 'link': 'https://www.youtube.com/watch?v=bslcx4LRFL0', 'description': 'No description available.', 'thumbnail': {'static': 'https://i.ytimg.com/vi/bslcx4LRFL0/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLCFuAHO-C59xHl81mc7YhsfZ8B3WQ', 'rich': 'https://i.ytimg.com/an_webp/bslcx4LRFL0/mqdefault_6s.webp?du=3000&sqp=CO6v6r4G&rs=AOn4CLBs0OS7DeHEvN5BqE32i9gHe6pkfQ'}, 'views': 14596945, 'length': '1:55:59', 'summary': "The video opens with a dramatic confrontation between two individuals, presumably in a romantic relationship, at 0:00. The atmosphere is tense, with one person, a woman, pleading with her partner for $2,000, emphasizing that it's not a significant amount, given what she knows about his financial situation. Her tone is desperate, and her body language conveys a sense of urgency and frustration. The lighting in the scene is dim, with a warm color palette that s